In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Preprocessing data
dataset_train = pd.read_csv('/content/drive/MyDrive/deep learning/deep_learning_a-z/Part 3 - Recurrent Neural Networks (RNN)/Google_Stock_Price_Train.csv')
training_set = dataset_train.iloc[:, 1:2].values

In [ ]:
# scale the data
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler(feature_range=(0, 1))

training_set_scaled = sc.fit_transform(training_set)

In [ ]:
# timesteps
X_train = []
y_train = []

for i in range(60, 1257):
  X_train.append(training_set_scaled[i - 60:i, 0])
  y_train.append(training_set_scaled[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [ ]:
# The build of the RNR
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

regressor = Sequential()

# First LSTM layer
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1) ))
regressor.add(Dropout(0.2))

# Second LSTM layer
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Third LSTM layer
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Quarter LSTM layer
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

# Add the fully connected layer
regressor.add(Dense(units = 1))

# Compile the model
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [ ]:
# Train the model
regressor.fit(X_train, y_train, epochs=100, batch_size=32)

In [ ]:
# Predict the values of january 2017
dataset_test = pd.read_csv('/content/drive/MyDrive/deep learning/deep_learning_a-z/Part 3 - Recurrent Neural Networks (RNN)/Google_Stock_Price_Test.csv')
real_stock_price = dataset_test.iloc[:, 1:2].values

# Prepare the data for doing the predictions
dataset_total = pd.concat((dataset_train['Open'], dataset_test['Open']), axis=0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1, 1)
inputs = sc.transform(inputs)
X_test = []
for i in range(60, 80):
  X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

In [ ]:
# Make the grafic to see the data
plt.plot(real_stock_price, color = 'red', label = 'Precio real de la accion')
plt.plot(predicted_stock_price, color = 'blue', label = 'Precio predicho de la accion')
plt.title('Prediccion con una red neuronal recurrente el precio de una accion de google')
plt.xlabel('Fecha')
plt.ylabel('Precio de la accion de google')
plt.show